In [5]:
# Import necessary libraries for the model
from music21 import converter, note, chord, stream, instrument # Converts MIDI files into useful data
from tensorflow.keras.models import Sequential # Chose this model just because haha
from tensorflow.keras.layers import LSTM, Dense, Dropout # LSTMs are apparantely great for music generation
from tensorflow.keras.utils import to_categorical # This converts class vectors (integers) into one-hot encoding which is great for classification
import numpy as np
import os
import glob # Library to find files with similar namings (ex. .mid files)

In [6]:
# Function to extract notes and chords from multiple MIDI files
def get_notes_from_directory(directory_path): # Takes MIDI files as input and returns a list of notes and chords
    notes = []
    midi_files = glob.glob(os.path.join(directory_path, "*.mid")) # Finds all .mid files and creates a file path

    for file in midi_files:
        midi = converter.parse(file) # Changes MIDI into music21 object
        parts = instrument.partitionByInstrument(midi) # Finds if there are multiple instruments in MIDI files
        if parts:
            notes_to_parse = parts.parts[0].recurse() # Selects first instrument and collects all notes
        else:
            notes_to_parse = midi.flat.notes # Just takes the all the notes

        for element in notes_to_parse: # Loops through each note taken from above
            if isinstance(element, note.Note): # Checks if it is a single note, if it is it adds it"s pitch
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord): # Checks if it is a chord, if it is it adds a list of pitches
                # Convert chord to string of its constituent notes
                notes.append(".".join(str(n) for n in element.normalOrder))
                
    return notes

midi_directory = "dataset"  # Replace with your directory path
notes = get_notes_from_directory(midi_directory)

In [7]:
# Data Preprocessing
sequence_length = 100  # Number of notes to be inputted into model

unique_notes = sorted(set(notes)) # Removes duplicate notes and sorts them
n_vocab = len(unique_notes) # Total number of unique notes/chords
note_to_int = dict((note, number) for number, note in enumerate(unique_notes)) # Creates a dictionary to map each note to an integer (Ex. 1, C#4 (probably not right lol))

network_input = []
network_output = []
for i in range(0, len(notes) - sequence_length):
    sequence_in = notes[i:i + sequence_length] # A slice from the notes, which will be used as input
    sequence_out = notes[i + sequence_length] # The note that the model will try to predict
    network_input.append([note_to_int[char] for char in sequence_in]) # Converts each note to its matching integer
    network_output.append(note_to_int[sequence_out]) # Converts output note to its matching integer

n_patterns = len(network_input)

# Reshape for LSTM input: [samples, time steps, features]
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1)) # Shapes the list into a 3D array to feed into TensorFlow)
network_input = network_input / float(n_vocab)  # Normalize to be between 0-1 (machine learning convention)

network_output = to_categorical(network_output, num_classes=n_vocab) # Convert output to one-hot encoding for easier classification

In [8]:
# Creating model
model = Sequential() # Initialize model
model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True)) # Manually add layers for the Neural Network
model.add(Dropout(0.3)) # This helps with overfitting by dropping 30% of the input to 0
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256, activation="relu")) # Adding a dense layer with ReLU for more complex learning
model.add(Dropout(0.3))
model.add(Dense(n_vocab, activation="softmax")) # Adds a probability distribution over all notes

d:\SolarCarProject\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
# Training the Model
model.compile(loss="categorical_crossentropy", optimizer="adam") # This adds a loss function for classification and algorithm called adam for weights
model.fit(network_input, network_output, epochs=100, batch_size=64) # Trains the model for 100 epochs (can adjust it since 100 epochs took me 1 hour)

Epoch 1/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - loss: 5.4101
Epoch 2/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - loss: 5.0413
Epoch 3/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - loss: 4.9777
Epoch 4/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - loss: 4.9568
Epoch 5/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - loss: 4.9175
Epoch 6/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - loss: 4.9888
Epoch 7/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - loss: 4.9602
Epoch 8/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - loss: 4.9754
Epoch 9/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - loss: 5.1488
Epoch 10/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - loss: 5.0052
Epoch 11/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - loss: 4.9538
Epoch 12/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - loss: 4.9428
Epoch 13/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - loss: 4.9500
Epoch 14/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - loss: 4.9698
Epoch 15/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - loss: 4.9671
Epoc

In [10]:
# Generate music with predicted notes
def generate_music(model, network_input, note_to_int, int_to_note, n_vocab, num_notes=500): 
    start = np.random.randint(0, len(network_input) - 1) # Select random starting point
    pattern = network_input[start] # Starting pattern
    prediction_output = []

    for note_index in range(num_notes):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1)) # Reshapes into shape required by model
        prediction = model.predict(prediction_input, verbose=0) # Uses model to predict next note
        index = np.argmax(prediction) # Finds the note with highest probability
        result = int_to_note[index] # Converts index to corresponding note
        prediction_output.append(result)
        pattern = np.append(pattern, index) # Updates pattern with predicted note
        pattern = pattern[1:len(pattern)]

    return prediction_output

# Create reverse mapping from integers to notes
int_to_note = dict((number, note) for number, note in enumerate(unique_notes))

# Generate a sequence of 500 notes
predicted_notes = generate_music(model, network_input, note_to_int, int_to_note, n_vocab, num_notes=500)

In [11]:
# Convert Predictions to MIDI
def create_midi(predicted_notes, output_path="output.mid"):
    offset = 0 # Offset for note start time
    output_notes = []

    for pattern in predicted_notes: # Loops through each note/chord
        if ("." in pattern) or pattern.isdigit(): # Checks if it is a chord. If so, splits it into individual notes
            notes_in_chord = pattern.split(".")
            notes = []
            for current_note in notes_in_chord: # Creates a new note from the chord and sets instrument to piano
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes) # Creates new chord from notes and sets it as piano
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:  # Checks if it is a note. If it is, creates a note object and sets instrument to piano
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        
        offset += 0.5 # Spacing between notes

    midi_stream = stream.Stream(output_notes) # Creates a stream object containing all notes
    midi_stream.write("midi", fp=output_path) # Creates midi file from stream object

create_midi(predicted_notes, output_path="generated_music.mid") # Save the generated music to a MIDI file

print("Success") # Prints success statement. MIDI file will be generated as generated_music.mid

Music generation complete. Check the 'generated_music.mid' file.
